![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/open-source-nlp/14.0.GPT2_Transformer_In_SparkNLP.ipynb)

# GPT2Transformer: OpenAI Text-To-Text Transformer

![image.png](https://miro.medium.com/max/1400/1*UZTblFMn8oWfmqBMQhUOdA.jpeg)

[GPT-2](https://openai.com/blog/better-language-models/)  displays a broad set of capabilities, including the ability to generate conditional synthetic text samples of unprecedented quality, where the model is primed with an input and it generates a lengthy continuation.

Pretrained models can be loaded with `pretrained()` of the companion object:

**GPU MODE** is highly reccomended, since this is a very large model.

## Colab Setup

In [ ]:
! pip install -q pyspark==3.4.1 spark-nlp==5.1.2

In [ ]:
import sparknlp

from sparknlp.base import *
from sparknlp.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel

**Make sure to Enable GPU Mode and High RAM**

In [ ]:
# Comment out this line  and uncomment the next one to enable GPU mode and High RAM
#
spark = sparknlp.start()

# spark = sparknlp.start(gpu=True)

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark

Spark NLP version 5.1.2
Apache Spark version: 3.4.1


**GPT2 Models In Spark NLP**

*   `gpt2`
*   `gpt2_medium`
*   `gpt2_distilled`
*   `gpt2_large`

The default model is `"gpt2"`, if no name is provided. For available pretrained models please see the [Spark NLP Models Hub](https://nlp.johnsnowlabs.com/models?q=gpt2)



## GPT2 Pipeline

Now, let's create a Spark NLP Pipeline with `gpt2_medium` model and check the results.

In [ ]:
documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("documents")

gpt2 = GPT2Transformer.pretrained("gpt2_medium") \
    .setInputCols(["documents"]) \
    .setMaxOutputLength(50) \
    .setMinOutputLength(25) \
    .setOutputCol("generation")

data = spark.createDataFrame([["My name is Leonardo."]]).toDF("text")

pipeline = Pipeline().setStages([documentAssembler,gpt2]).fit(data)

result = pipeline.transform(data)
result.select("generation.result").show(truncate=False)

gpt2_medium download started this may take some time.
Approximate size to download 1.2 GB
[OK!]
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|result                                                                                                                                                                                                                                           |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[ My name is Leonardo. I'm a student at the University of California, Berkeley. I've been studying computer science for the past two years. I have a PhD in computer scienc

We can display the documentation of all params with their optionally default values and user-supplied values by `explainParams()` function

In [ ]:
print(gpt2.explainParams())

batchSize: Size of every batch (default: 4)
configProtoBytes: ConfigProto from tensorflow, serialized into byte array. Get with config_proto.SerializeToString() (undefined)
doSample: Whether or not to use sampling; use greedy decoding otherwise (default: False, current: False)
engine: Deep Learning engine used for this model (default: tensorflow)
ignoreTokenIds: A list of token ids which are ignored in the decoder's output (default: [])
inputCols: previous annotations columns, if renamed (current: ['documents'])
lazyAnnotator: Whether this AnnotatorModel acts as lazy in RecursivePipelines (default: False)
maxOutputLength: Maximum length of output text (default: 20, current: 50)
minOutputLength: Minimum length of the sequence to be generated (default: 0, current: 25)
noRepeatNgramSize: If set to int > 0, all ngrams of that size can only occur once (default: 0, current: 3)
outputCol: output annotation column. can be left default. (current: generation)
repetitionPenalty: The parameter for

Let's use model with more sentences and set `.setDoSample()` parameter as True, this parameter is used for whether or not to use sampling; use greedy decoding otherwise, by default False. <br/>
Also, we use `.setTopK()` parameter for the number of highest probability vocabulary tokens to keep for top-k-filtering, by default 50.

In [ ]:
sample_texts= [[1, "Mey name is  Leonardo"],
               [2, "My name is Leonardo and I come from Rome."],
               [3, "My name is"],
               [4, "What is the difference between diesel and petrol?"]]

sample_df= spark.createDataFrame(sample_texts).toDF("id", "text")

In [ ]:
gpt2.setMaxOutputLength(50).setMinOutputLength(25).setDoSample(True).setTopK(20)

GPT2TRANSFORMER_729a8e69e804

In [ ]:
result = pipeline.transform(sample_df)
result.select("id", "generation.result").show(truncate=False)

+---+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id |result                                                                                                                                                                                                                                                   |
+---+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1  |[ Mey name is  Leonardo Da Vinci. So you can guess who he worked with? Da Vincies friend and fellow artist. He actually used his own model of Da Vince's " The Magic Mirror " to create this piece.  ]                             

### Changing the Transformer's task

Now, we change the task of Transformer. We can verify some informations to GPT-2 by setting `.setTask()` parameter as **"Is it true that"**. <br/>
We give a text to the model by setting `setTask("Is it true that")` and model adds the "Is it true that" expression at the beginning of the sentence and generates sentences.

In [ ]:
gpt2.setTask("Is it true that").setMaxOutputLength(50)

GPT2TRANSFORMER_729a8e69e804

In [ ]:
sample_text= [[1, "Donald Trump is rich?"],
              [2, "Pink Floyd is rock band?"]]

sample_df= spark.createDataFrame(sample_text).toDF("id", "text")

result = pipeline.transform(sample_df)
result.select("id", "generation.result").show(truncate=False)

+---+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id |result                                                                                                                                                                                                                                     |
+---+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1  |[ Is it true that Donald Trump is rich? (Answer: Yes, but only because he can buy everything and everything is expensive.) Is it possible for Donald Trump to become president of the United States? (Yes, but if he is elected then he is]|
|2  |[ Is it true that Pink Floy

# Some Helper Methods for generating and printing generation results

In [ ]:
from typing import List
def prompts_to_spark_df(prompts,spark=spark):
  text = [[i, prompt] for i,prompt in enumerate(prompts)]
  return spark.createDataFrame(text).toDF("id", "text")

def generate_with_gpt2(gpt_pipe, prompts,task='generate', print=True):
  if isinstance(prompts,str):
    df = prompts_to_spark_df([prompts])
  elif isinstance(prompts,List):
    df = prompts_to_spark_df(prompts)
  else :
    raise ValueError(f"Invalid Type = {type(prompts)} please pass a str or list of str for prompts parameter ")
  df = pipeline.transform(df)
  df = df.select("id",'text', "generation.result").toPandas()

  if print:
    print_generation_results(df)
  return df

def print_generation_results(df):
  for idx, row in df.iterrows():
    print(f'Example {idx}: {200*"_"}')
    print(row.result[0])
    print('\n')


# Explore GPT Parameters Play with Paramns

### Sampling Methods


Sampling means we **randomly** draw from a distribution of words.
The probability distribution is conditioned on all previous tokens in a text to generate the next token.

By default the distribution contains all words in the vocabulary of GPT2, where many candidates are incorrect to generate.

There are two methods of reshaping and drawing from those distributions :

1. **Top-K Sampling** Take the k most likely words from the original distribution. Redistribute probability mass among those k words and draw according to the new probabilities.

2. **Top-P Nucleus sampling**  Take smallest possible set of N words, which  together have a probability of p. Redistribute probability mass among those N words and draw according to the new probabilities.



Additionally, both methods can be tweaked ith the following parameters :

- **temperature** : Parameter of the softmax function which affect the distrubtion computed by the model. The closer we are to 0, the more deterministic the probability will become, distribution tails will become slimmer and outlier word probabilites are more close to 0. Temperature values closer values to 1 make tails of probability fatter which makes outliers more probable and generic results less probable.


These parameters are shared by all method :
- **ignoreTokenIds**: A list of token ids which are ignored in the decoder's output (default: [])
- **noRepeatNgramSize**: If set to int > 0, all ngrams of that size can only occur once
- **repetitionPenalty**: The parameter for repetition penalty. 1.0 means no penalty.  https://arxiv.org/pdf/1909.05858.pdf>
- **task**:  Transformer's task, e.g. 'is it true that'> (default: , current: generate)

In [ ]:
print(gpt2.explainParams())

batchSize: Size of every batch (default: 4)
configProtoBytes: ConfigProto from tensorflow, serialized into byte array. Get with config_proto.SerializeToString() (undefined)
doSample: Whether or not to use sampling; use greedy decoding otherwise (default: False, current: True)
engine: Deep Learning engine used for this model (default: tensorflow)
ignoreTokenIds: A list of token ids which are ignored in the decoder's output (default: [])
inputCols: previous annotations columns, if renamed (current: ['documents'])
lazyAnnotator: Whether this AnnotatorModel acts as lazy in RecursivePipelines (default: False)
maxOutputLength: Maximum length of output text (default: 20, current: 50)
minOutputLength: Minimum length of the sequence to be generated (default: 0, current: 25)
noRepeatNgramSize: If set to int > 0, all ngrams of that size can only occur once (default: 0, current: 3)
outputCol: output annotation column. can be left default. (current: generation)
repetitionPenalty: The parameter for 

### Play with temperature
Set Temperature higher to make GPT more random/creative and text less coherent
Temperature > 0  and Temperature <=1
You must set `gpt2.setDoSample(True)` to have non-deterministic results

In [ ]:
text = """Hello my name is GPT2, I love to """
data = [text, text,text,text,text ]
gpt2.setMaxOutputLength(200)

GPT2TRANSFORMER_729a8e69e804

In [ ]:
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, data,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to !!! I can't wait to have my wife be so happy that she knows me as her GPT1! I want her to know that we are still together and that she deserves what she got. If only people could have understood and understood how much my daughter needs. The love, support, and happiness I feel for my GPT are just a couple of thoughts from my own heart, but this is the true picture I see.


Example 1: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to ive made a video in english.I like to go from video to video to see a change in skin color and i do 

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
1,1,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
2,2,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
3,3,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
4,4,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."


In [ ]:
gpt2.setTemperature(0.75)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, data ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to !!! But I am scared that I can't use the password! Thank you in advance.


Example 1: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to !!!

GPT2 on Instagram!

My name is Raul and I am an actor from Argentina. I'm 28 years old and I'm currently playing as the character of GPT1 in a very popular series of animated series. I am also a lover of cooking and I love cooking with my friends and family. My favorite thing is to go to the beach with my girlfriend and have a nice time together.

How did you get started in acting?

I was alwa

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
1,1,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
2,2,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
3,3,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
4,4,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."


In [ ]:
gpt2.setTemperature(0.5)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, data,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to  read manga and  I'm a big manga fan and I've been reading a lot of manga lately. I've started reading the manga series called "Hiroshima" by the author, Hidetaka "Hidetoshi" Taniguchi. I read the manga because I love manga, and I'm a huge fan of anime. I like to watch anime and manga, so I'm looking forward to reading this series. I'm also a big fan of manga and I think that this manga series is really good. I think it has great characters, and a lot about the characters are really interesting. I really like the story, and the characters, so this is the manga I'm going to read.
I think it's really good that you can read the series and read it in this format. I want to read this manga in this style. I would like to read it with


,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
1,1,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
2,2,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
3,3,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
4,4,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."


In [ ]:
gpt2.setTemperature(0.25)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline,data,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to !!!

I have been playing with the GPT 2 for a while now, and I am very happy with it. It is very easy to use, and it is very fast. I have been using it for about 2 weeks now, so I have a lot of experience with it, and am very satisfied with it so far. I am using it on my laptop, and my phone is also using it. I can also use it on the PC, and the phone is using it too. I don to use the phone as a phone, because I don't want to lose my data connection. I also don't use the PC as a PC, because it is too slow. I use the GPG key to sign the files, and then I can use the computer as a computer. I will be using the GPM to sign files, so that I can sign them. I do not have to worry


Example 1: __________________________________________

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
1,1,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
2,2,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
3,3,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
4,4,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."


In [ ]:
gpt2.setTemperature(0.1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, data,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to !!! But is it true? Can I install some packages of Debian 8 by a community software to work my Ubuntu/Mac systems from here? It is, this site also offer, in general, several of its projects under license GNU GPLv3. Are you in trouble as GPT3 version and all software with similar or older copyright will be rejected at the last because of different code? So in addition to install Linux software of its Debian equivalent as I need only do something more serious I recommend Linux Debian to do my need because we could build new life without software restrictions by use of some third party toolchain of free/open source Software . The question on here might be very tricky when my main motivation from here is to keep with our freedom wit

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
1,1,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
2,2,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
3,3,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
4,4,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."


In [ ]:
gpt2.setTemperature(0.0001)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, data ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to ________________, HiGTPiRzCq4Nc7H3R7c1jdPtPX9P5p4x
It does however work just okay so its definitely more like how an eee can feel with gppa . Not great yet for someone to install from usb though if only one guy or so gets hold in him!So that might ormay explain, just so no surprises:My eemup does appear better however...it appears slightly weaker dueto what a bunch nuke said that we already already found here as stated a previous week..If they will keep talking then more testing are surely being ordered soon enoughI've made more sure, thanks very kindly people are interested :),Hope everything go very right this new development cycle in linux . This project also deserves our applause if i'll continue its working !It just got ver

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
1,1,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
2,2,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
3,3,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
4,4,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."


# GPT2 Use Cases

These are some examples to give you an idea of what GPT2 is capable of

## GPT for Programming Code

### GPT and Scala

In [ ]:
# Hello world in Scala
scala_code = """
object Hello {
    def main(args: Array[String]) = {
        println("Hello, world")
    }
}
"""
gpt2.setMaxOutputLength(1000)

gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, scala_code ,task='generation', print=True)



Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
object Hello {
    def main(args: Array[String]) = {
        println("Hello, world")
    }
}
I'm going to start with the simplest example of Hello , and then see that you can write some more complex ones.


Let's start with one simple Hello :

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 class Hello { public : void main() { Hello   = 123456789; } private : int Hello; public : static Hello hello; }; class Foo { public: Foo () {} void Hello() { greet(1, Hello) ; } };


And then some more complicated ones:

class SomeClass { static Hello greeting(){ Hello hello = greeting.get( 12345 ); } void greet(int x, String s) { greet("Hello", x); } void Hello(int a, String b) { hello("hello", b

,id,text,result
0,0,\nobject Hello {\n def main(args: Array[Str...,[ Is it true that \nobject Hello {\n def ma...


### GPT and Python

In [ ]:
# adding 3 numbers in python
py_code = """
def my_func(a,b,c):
  return a+b+c
"""
gpt2.setMaxOutputLength(1000)

gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, py_code ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
def my_func(a,b,c):
  return a+b+c

# the argument is an instance variable

def instance_method(my_func, *args, **kwds):

" This is the "method" to call and return. "

for kwds in my_method's:

kwds.pop(kwds)

return kwda

class defmethod( *arg, **args): "
 the method to call, or return a dict of arguments and return values

(this will return an

instance "method".) "
 [ 1 ]

methods = [

my_method ( *arg),

_method,

__method, # optional method name or a single keyword,
.get,
 1 , 1 ,

0 , 3 ,
 ' foo ' , # optional function parameter

]

@ property

public function name ( kw ): " Name of the method used to create an instance instance of this method. " """

from sys import argname, argtypes, argname_types, _, getattr

pass

get_instance = __main__




,id,text,result
0,0,"\ndef my_func(a,b,c):\n return a+b+c\n","[ Is it true that \ndef my_func(a,b,c):\n ret..."


In [ ]:
# breadth-first search
bfs_python_code = """
def bfs(visited, graph, node):
  visited.append(node)
  queue.append(node)

  while queue:
    s = queue.pop(0)
    print (s, end = " ")

    for neighbour in graph[s]:
      if neighbour not in visited:
        visited.append(neighbour)
        queue.append(neighbour)
"""


gpt2.setMaxOutputLength(1000)
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, bfs_python_code ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
def bfs(visited, graph, node):
  visited.append(node)
  queue.append(node)

  while queue:
    s = queue.pop(0)
    print (s, end = " ")

    for neighbour in graph[s]:
      if neighbour not in visited:
        visited.append(neighbour)
        queue.append(neighbour)
# The order of events is important for the graph
# function. The nodes get the first update, the queue receives the second, the visited object is the last update
# and the graph gets the rest of the updates.


def graph[node, count](event: Event): List<Event>:

# This function needs to return a List in order:

" -------------- ------------- --------------

(Event.get(0), event.get(-0), Event.get((event.get(*count, 0))), event)

"""

def count(): return list([event])

'''

returns a List of event

,id,text,result
0,0,"\ndef bfs(visited, graph, node):\n visited.ap...","[ Is it true that \ndef bfs(visited, graph, no..."


## Generate Product Recommendations

We can start a base list of top 10 anything and GPT2 will complete it for us!      
This is very useful to reccomendation and search engines.

### Top 10 Movie List

In [ ]:
movie_list = """My top 10 movie list :
1. The Matrix 1
2. The Terminator 1
3. Scarface
4."""

gpt2.setTemperature(1)
gpt2.setMaxOutputLength(200)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, movie_list ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that My top 10 movie list :
1. The Matrix 1
2. The Terminator 1
3. Scarface
4. The Lord of the Rings: The Fellowship of the Ring
5. The Shining 2
6. Terminator 2
7. The Dark Knight Rises
8. Raiders of the Lost Ark
9. King Kong II
10. Lord of War, The (2009)
My ranking order of my "top 10 movies."
1) My top 5 movie list:  The Matrix: The Complete Film Trilogy.
2) My 10 favorite movies of all time:  Terminator Salvation, Terminator Salvation 2: Judgment Day and Terminator Salvation 3: Rise of the Machines.
3) Best TV series/movies (all time):  Breaking Bad, Game of Thrones, Sons of Anarchy, The Walking Dead, Breaking Bad 2: Better Call Saul, Mad Men, Lost.
4) My favorite TV series:  CSI: Crime Scene Investigation




,id,text,result
0,0,My top 10 movie list :\n1. The Matrix 1\n2. Th...,[ Is it true that My top 10 movie list :\n1. T...


How **not** to generate text with GPT2

Since every new word is conditioned on every previous **character**, things like **white spaces** and **new lines** can skew the sampling distribution into an unintended direction

In [ ]:
# NOTE! We have malformed the input text and added a newline.
# This will Confuse GPT2, it might think the list is over and probability distribution changes accordingly
movie_list = """My Top 10 movie list :
1. The Matrix 1
2. The Terminator 1
3. Scarface
4.

"""
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, movie_list ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that My Top 10 movie list :
1. The Matrix 1
2. The Terminator 1
3. Scarface
4.


I am always going through lists of movies that I think should be on my list because I see them all. I guess I also like to think that I am smarter than everyone else, or at least, that if I write a list of my favorite movies, it is more likely that it will be true. I don't know how to quantify this, so I will go for the simplest answer:

1 and 2 are good movies because their plots are not so predictable

And 3 is great because it is not predictable and they have some good characters

I'm sure you can agree or disagree with me in this, but my list does not include any of the bad movies that made people think that The Matrix was a terrible movie. I do not feel that that movie should have 

,id,text,result
0,0,My Top 10 movie list :\n1. The Matrix 1\n2. Th...,[ Is it true that My Top 10 movie list :\n1. T...


In [ ]:
movie_list = """My Top 10 worst movie list :
1. The Matrix 4
2. Jack and Jill
3. Super Mario Bros. (1993)
4. """
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, movie_list ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that My Top 10 worst movie list :
1. The Matrix 4
2. Jack and Jill
3. Super Mario Bros. (1993)
4.  The Shining 5
5.  A Most Violent Year
6. Â The Goonies
7.    Hitchhiker's Guide to the Zombie Apocalypse     8.  Â Back to the Future (1989)
9. 
The Terminator 8.
10.  Batman
A friend sent in this great list!
So, that's what the list looks like! I love this list! The most amazing part about the list is the different movies each person liked/dislike, not just based on their top ten. Also, there is no ranking system for the movie. So if you see that one movie you really like and hate, that is awesome! So that's about it for this month guys, if you have any other movie or TV series you can




,id,text,result
0,0,My Top 10 worst movie list :\n1. The Matrix 4\...,[ Is it true that My Top 10 worst movie list :...


### Top 10 Game List

In [ ]:
game_list = """My Top 10 video games of all time list :
1. Half-Life 2
2. Super Smash Bros Melee
3. Portal
4. """
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, game_list ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that My Top 10 video games of all time list :
1. Half-Life 2
2. Super Smash Bros Melee
3. Portal
4.  Call of Duty Black Ops 3
5. Final Fantasy VII
6. Dragon's Lair
7.  The Legend of Zelda:
"What's your top 10 list ? What was the hardest thing about that?"
"I'm not sure, actually. I'm just thinking about how hard it is to make games like that. That's how frustrating I feel."
"So, it would probably be about how difficult it was to bring it all together in making The Legend Of Zelda: Breath Of The Wild. I'd imagine it's a little bit easier for me to create a game of that scope if that's the case..."
I guess this is not surprising. You're probably thinking "how could someone so skilled in video games have to go through that pain, that is so hard on their own




,id,text,result
0,0,My Top 10 video games of all time list :\n1. H...,[ Is it true that My Top 10 video games of all...


### Top 10 Book List

In [ ]:
game_list = """My Top 10 books of all time list :
1. 1964
2. Moby Dick
3. robinson crusoe
4. """
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, game_list ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that My Top 10 books of all time list :
1. 1964
2. Moby Dick
3. robinson crusoe
4.  Fairy tales
5. "Cabaret"
6. Dostoyevsky
7. The Adventures of Huckleberry Finn
8. 
9.  The Lion's Mane
10.  Hamlet
10) How does a book about how to have children stack up against the books about what to do when you get divorced? I think the answer is: It's both, and it's the one of the two that I have in the top 10.
My Top 10 Books of All Time




,id,text,result
0,0,My Top 10 books of all time list :\n1. 1964\n2...,[ Is it true that My Top 10 books of all time ...


## SEO Text / Content Writing / Copy Writing
Lets generate some texts for our new imaginary shopify store!
We will sell some hoodies, soap and beard products.

Based on some base templates, we can generate high quality marketing/seo texts

In [ ]:
# from https://www.shopify.com/blog/8211159-9-simple-ways-to-write-product-descriptions-that-sell

# Lets define some base texts, from which we will generate new texts

hoodie = """Can’t stop buying plants? Unbeleafable. Don’t worry—us too! Cover yourself in your favourite obsession in our NEW I Love Plants Oodie! """

soap  = """Made with real pine extract, this all-star bar is as tough as a freshly cut bat.
A true MVP of the shower, this heavy-hitter knocks out grime with its gritty composition and ultra-manly, woodsy scent."""

beard = """ Whatever your style is, Beardbrand Styling Balm is versatile enough to handle it.
Designed to work with all hair types, it provides enough hold to keep thick, curly hair under control, """
gpt2.setTemperature(1)
gpt2.setDoSample(True)

GPT2TRANSFORMER_729a8e69e804

#### Marketing Texts for Hoodies

In [ ]:
# Lets generate some marketing texts for our hoodies
generate_with_gpt2(pipeline, [hoodie,hoodie,hoodie,hoodie,hoodie,hoodie] ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Cant stop buying plants? Unbeleafable. Dont worryus too! Cover yourself in your favourite obsession in our NEW I Love Plants Oodie!  Here's our NEW BONUS: our NEW CUSTOMER APP!  
We will have a NEW and NEW!
The first and most popular customer service ever! We will NEVER have an issue with someone else and they always have my back! You don't want to be in a position where we aren't going to get your order! So please keep sending us your orders as soon as they're received so we can take care of them ASAP!


Example 1: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Cant stop buying plants? Unbele

,id,text,result
0,0,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Is it true that Cant stop buying plants? Unb...
1,1,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Is it true that Cant stop buying plants? Unb...
2,2,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Is it true that Cant stop buying plants? Unb...
3,3,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Is it true that Cant stop buying plants? Unb...
4,4,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Is it true that Cant stop buying plants? Unb...
5,5,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Is it true that Cant stop buying plants? Unb...


#### Marketing Texts for Soap

In [ ]:
# Lets generate some marketing texts for our soap
generate_with_gpt2(pipeline, [soap,soap,soap,soap,soap,soap] ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Made with real pine extract, this all-star bar is as tough as a freshly cut bat.
A true MVP of the shower, this heavy-hitter knocks out grime with its gritty composition and ultra-manly, woodsy scent.
Coconut soap is not one of our favorites. Not the most popular among our baristas. But these are the kind of products they need.
These are the sort of products that make people like us, that we're able to work with. These are the sorts of brands we use when we're out to buy something new.
It's the type of product we want out. That the people who make it want out too. We are very grateful for this support and thank you all for your kind comments. If we can find a way to get more people involved with this, we could really do some good for our barista community.


Ex

,id,text,result
0,0,"Made with real pine extract, this all-star bar...","[ Is it true that Made with real pine extract,..."
1,1,"Made with real pine extract, this all-star bar...","[ Is it true that Made with real pine extract,..."
2,2,"Made with real pine extract, this all-star bar...","[ Is it true that Made with real pine extract,..."
3,3,"Made with real pine extract, this all-star bar...","[ Is it true that Made with real pine extract,..."
4,4,"Made with real pine extract, this all-star bar...","[ Is it true that Made with real pine extract,..."
5,5,"Made with real pine extract, this all-star bar...","[ Is it true that Made with real pine extract,..."


#### Marketing Texts for beard products

In [ ]:
# Lets generate some marketing texts for our beard products
generate_with_gpt2(pipeline, [beard,beard,beard,beard,beard,beard] ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that  Whatever your style is, Beardbrand Styling Balm is versatile enough to handle it.
Designed to work with all hair types, it provides enough hold to keep thick, curly hair under control,  or straight hair on the back for a healthy look. Beardbrand's innovative, natural styling products are made with real ingredients with the highest quality ingredients available. Whether your hair type is straight or curly, this product will create natural hair growth, not to mention, feel great, no matter what your hair style is.
If your hair needs a bit of extra conditioning, this is the product for you. The scent that we create with this product is very unique. We use essential oils and natural fragrances throughout this product. It gives you a fresh scent that is both calmin

,id,text,result
0,0,"Whatever your style is, Beardbrand Styling Ba...","[ Is it true that Whatever your style is, Bea..."
1,1,"Whatever your style is, Beardbrand Styling Ba...","[ Is it true that Whatever your style is, Bea..."
2,2,"Whatever your style is, Beardbrand Styling Ba...","[ Is it true that Whatever your style is, Bea..."
3,3,"Whatever your style is, Beardbrand Styling Ba...","[ Is it true that Whatever your style is, Bea..."
4,4,"Whatever your style is, Beardbrand Styling Ba...","[ Is it true that Whatever your style is, Bea..."
5,5,"Whatever your style is, Beardbrand Styling Ba...","[ Is it true that Whatever your style is, Bea..."


## Song Lyrics

#### Last Christmas

In [ ]:
last_christmas = """
Last Christmas, I gave you my heart
But the very next day, you gave it away
This year, to save me from tears
I'll give it to someone special
"""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, last_christmas,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
Last Christmas, I gave you my heart
But the very next day, you gave it away
This year, to save me from tears
I'll give it to someone special
Last year, my life would have ended
With just one thought in your mind
Now the pain will end
With only that little thought in mind
And in time the pain won't end
I'm a fighter
A champion
It's over, no one will win
The game was won
But it didn't matter to me
So I'll be there every day to see you fall
The pain is finally over
My name is
But I will always be here, I'll always be there I won't let anyone take this away I'll never lose hope
This way I won (with my own sword) I'll fight to the bitter end
This is my life I don't care if you hate me or not
I will keep going with my heart, with my life even
The blood you have on y

,id,text,result
0,0,"\nLast Christmas, I gave you my heart\nBut the...","[ Is it true that \nLast Christmas, I gave you..."


#### Fresh prince of GPT

In [ ]:
bel_air = """
Now, this is a story all about how
My life got flipped-turned upside down
And I'd like to take a minute
Just sit right there
I'll tell you how I became the prince of a town called Bel-Air
"""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, bel_air,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
Now, this is a story all about how
My life got flipped-turned upside down
And I'd like to take a minute
Just sit right there
I'll tell you how I became the prince of a town called Bel-Air
I was born in a town known
As 'Bel-Air'
It's famous as a place where
A prince from the future could come
And live and die on his own
That's right, that's how I got my name and it was called 'Papa'
And the reason that I'm calling myself that, was that I had a mother named Puck
And her name was Pucky And I was born
And grew up in a house with a lot of toys
And a lot
There was a lady in that house, whose name is
It must be that woman who's called
"the witch"
Pucky came to the house when
She was eight years old and she was
A young girl, with a face
So pretty, that I thought you w

,id,text,result
0,0,"\nNow, this is a story all about how\nMy life ...","[ Is it true that \nNow, this is a story all a..."


#### GPT Jackson

In [ ]:
thriller_lyrics = """
They told him, "Don't you ever come around here"
"Don't wanna see your face, you better disappear"
The fire's in their eyes and their words are really clear
So beat it, just beat it
You better run, you better do what you can
Don't wanna see no blood, don't be a macho man
You wanna be tough, better do what you can
So beat it, but you wanna be bad
Just beat it (beat it), beat it (beat it)
No one wants to be defeated
Showin' how funky and strong is your fight
It doesn't matter who's wrong or right
Just beat it (beat it)
Just beat it (beat it)
"""


gpt2.setMaxOutputLength(500)
generate_with_gpt2(pipeline, thriller_lyrics,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
They told him, "Don't you ever come around here"
"Don't wanna see your face, you better disappear"
The fire's in their eyes and their words are really clear
So beat it, just beat it
You better run, you better do what you can
Don't wanna see no blood, don't be a macho man
You wanna be tough, better do what you can
So beat it, but you wanna be bad
Just beat it (beat it), beat it (beat it)
No one wants to be defeated
Showin' how funky and strong is your fight
It doesn't matter who's wrong or right
Just beat it (beat it)
Just beat it (beat it)
The sky is burning, your blood is flowing
You know, they ain't gonna let you down (beat your shit)




,id,text,result
0,0,"\nThey told him, ""Don't you ever come around h...","[ Is it true that \nThey told him, ""Don't you ..."


#### Real Slim GPT

In [ ]:
real_slim_shady = """
May I have your attention, please?
May I have your attention, please?
Will the real Slim Shady please stand up?
I repeat, will the real Slim Shady please stand up?
We're gonna have a problem here?
Y'all act like you never seen a white person before
Jaws all on the floor like Pam, like Tommy just burst in the door
And started whoopin' her ass worse than before
They first were divorced, throwin' her over furniture (ah!)
It's the return of the, oh wait, no way, you're kidding
He didn't just say what I think he did, did he?
And Dr. Dre said-, nothing you idiots
Dr. Dre's dead, he's locked in my basement (haha)
"""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, real_slim_shady,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
May I have your attention, please?
May I have your attention, please?
Will the real Slim Shady please stand up?
I repeat, will the real Slim Shady please stand up?
We're gonna have a problem here?
Y'all act like you never seen a white person before
Jaws all on the floor like Pam, like Tommy just burst in the door
And started whoopin' her ass worse than before
They first were divorced, throwin' her over furniture (ah!)
It's the return of the, oh wait, no way, you're kidding
He didn't just say what I think he did, did he?
And Dr. Dre said-, nothing you idiots
Dr. Dre's dead, he's locked in my basement (haha)
And we all know there's a new doctor coming out with a new name (sigh)
A brand new face, I'm a fan now (gasp)
You should come check out this new doctor, he 

,id,text,result
0,0,"\nMay I have your attention, please?\nMay I ha...","[ Is it true that \nMay I have your attention,..."


#### GPT Rap God Bot

In [ ]:
rap_god_lyrics = """
Look, I was gonna go easy on you not to hurt your feelings
But I'm only going to get this one chance (six minutes-, six minutes-)
Something's wrong, I can feel it (six minutes, Slim Shady, you're on!)
Just a feeling I've got, like something's about to happen, but I don't know what
If that means what I think it means, we're in trouble, big trouble
And if he is as bananas as you say, I'm not taking any chances
You are just what the doc ordered
I'm beginnin' to feel like a Rap God, Rap God
All my people from the front to the back nod, back nod
Now, who thinks their arms are long enough to slap box, slap box?
They said I rap like a robot, so call me Rap-bot """

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, rap_god_lyrics,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
Look, I was gonna go easy on you not to hurt your feelings
But I'm only going to get this one chance (six minutes-, six minutes-)
Something's wrong, I can feel it (six minutes, Slim Shady, you're on!)
Just a feeling I've got, like something's about to happen, but I don't know what
If that means what I think it means, we're in trouble, big trouble
And if he is as bananas as you say, I'm not taking any chances
You are just what the doc ordered
I'm beginnin' to feel like a Rap God, Rap God
All my people from the front to the back nod, back nod
Now, who thinks their arms are long enough to slap box, slap box?
They said I rap like a robot, so call me Rap-bot 
I'll let it be Rap-drum, rap drum, Rap-rap, rap rap
It ain't gonna be no problem, I ain't going to lose my 

,id,text,result
0,0,"\nLook, I was gonna go easy on you not to hurt...","[ Is it true that \nLook, I was gonna go easy ..."


## Fiction Writing

#### Fantasy Stories

In [ ]:
unicorn_story = """In a shocking finding, scientist discovered a herd of unicorns living in a remote,
previously unexplored valley, in the Andes Mountains.
Even more surprising to the researchers was the fact that the unicorns spoke perfect English."""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, unicorn_story,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that In a shocking finding, scientist discovered a herd of unicorns living in a remote,
previously unexplored valley, in the Andes Mountains.
Even more surprising to the researchers was the fact that the unicorns spoke perfect English.
'They were extremely intelligent, with an average IQ of 90, which was higher than that of any other species in the world.'
According to the results, the researchers believe that 'The unicorns live in an extinct, isolated valley, known as the "Valley of the Sun" - an area of the Andean Mountains
It seems that these unique animals are very similar in morphology to humans, with large ears, thick coat and tails.
And the unicorn's tongue, as well as its teeth - which have large pointed ends - resemble those of a bear
Experts have concluded

,id,text,result
0,0,"In a shocking finding, scientist discovered a ...","[ Is it true that In a shocking finding, scien..."


In [ ]:
silly_story = """
Blue creatuers have been found in underground cavern in Maxico, which are mysteriously filled with peanut butter pockets.
These pockets are rich in proteins and enabled a civilization of these creatures to flurish and
build technologies which have surpassed human understanding. Scientist have studied these
creataures and found incredible discoveries, including their mastery of the Spanish language and perfect Tacos!"""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, silly_story,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
Blue creatuers have been found in underground cavern in Maxico, which are mysteriously filled with peanut butter pockets.
These pockets are rich in proteins and enabled a civilization of these creatures to flurish and
build technologies which have surpassed human understanding. Scientist have studied these
creataures and found incredible discoveries, including their mastery of the Spanish language and perfect Tacos!
This discovery of super-intelligence in a species of creatures has caused a great concern among the public and governments.  The
Scientists believe the discovery is a sign of a major problem that exists within mankind, and that we  have not adequately addressed the
disruptive powers of the  creatu that have been discovered in this area, and they ar

,id,text,result
0,0,\nBlue creatuers have been found in undergroun...,[ Is it true that \nBlue creatuers have been f...


#### Lovecraftian Horror Stories

In [ ]:
lovecraft_text = """
At this horror I sank nearly to the lichened earth,
transfixed with a dread not of this nor any world, but only of the mad spaces between the stars.

Out of the unimaginable blackness beyond the gangrenous glare of that cold flame,
out of the Tartarean leagues through which that oily river rolled uncanny, unheard, and unsuspected,
there flopped rhythmically a horde of tame, trained, hybrid winged things that no sound eye could ever wholly grasp,
or sound brain ever wholly remember. They were not altogether crows, nor moles, nor buzzards, nor ants,
nor vampire bats, nor decomposed human beings; but something I cannot and must not recall."""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, lovecraft_text,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
At this horror I sank nearly to the lichened earth,
transfixed with a dread not of this nor any world, but only of the mad spaces between the stars.

Out of the unimaginable blackness beyond the gangrenous glare of that cold flame,
out of the Tartarean leagues through which that oily river rolled uncanny, unheard, and unsuspected,
there flopped rhythmically a horde of tame, trained, hybrid winged things that no sound eye could ever wholly grasp,
or sound brain ever wholly remember. They were not altogether crows, nor moles, nor buzzards, nor ants,
nor vampire bats, nor decomposed human beings; but something I cannot and must not recall.
I am the son of that man who has died in the great fire, and in the place where he stood by me

that day, my father had died.

,id,text,result
0,0,\nAt this horror I sank nearly to the lichened...,[ Is it true that \nAt this horror I sank near...


In [ ]:
cthulu_intro = """
The most merciful thing in the world, I think, is the inability of the human mind to correlate all its contents.
We live on a placid island of ignorance in the midst of black seas of infinity, and it was not meant that we should voyage far.
The sciences, each straining in its own direction, have hitherto harmed us little"""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, cthulu_intro,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
The most merciful thing in the world, I think, is the inability of the human mind to correlate all its contents.
We live on a placid island of ignorance in the midst of black seas of infinity, and it was not meant that we should voyage far.
The sciences, each straining in its own direction, have hitherto harmed us little by little, until now the tide has been turned. Now that the tides are at a certain point, we may say with an eye of wonder: "Why should we venture farther into the dark, and farther from shore?" And if the answer is that we have found more of the same on a smaller or larger scale, we would have been better off without the sciences, except perhaps to seek the fountain from which the water rushes up again. It would not have happened unless the s

,id,text,result
0,0,"\nThe most merciful thing in the world, I thin...",[ Is it true that \nThe most merciful thing in...


#### Re-Write movie scripts

In [ ]:
the_matrix_script = """
The Architect: Hello, Neo.

Neo: Who are you?

The Architect: I am the Architect. I created the matrix. I've been waiting for you.
You have many questions, and although the process has altered your consciousness, you remain irrevocably human.
Ergo, some of my answers you will understand, and some of them you will not.
Concordantly, while your first question may be the most pertinent, you may or may not realize it is also irrelevant.

Neo: Why am I here?

The Architect: """

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, the_matrix_script,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
The Architect: Hello, Neo.

Neo: Who are you?

The Architect: I am the Architect. I created the matrix. I've been waiting for you.
You have many questions, and although the process has altered your consciousness, you remain irrevocably human.
Ergo, some of my answers you will understand, and some of them you will not.
Concordantly, while your first question may be the most pertinent, you may or may not realize it is also irrelevant.

Neo: Why am I here?

The Architect:  You need to understand that your matrix, or reality, has been altered. Your life will not be the same as it's been before. It may be better, or worse, but not as it once was. But it cannot be otherwise.
In order for you to understand what's behind your existence, it will first have to have been

,id,text,result
0,0,"\nThe Architect: Hello, Neo.\n\nNeo: Who are y...","[ Is it true that \nThe Architect: Hello, Neo...."


In [ ]:
the_matrix_script = """
The Architect: Hello, Neo.
Neo: Who are you?

The Architect: I am the Architect. I created the matrix. I've been waiting for you.
You have many questions, and although the process has altered your consciousness, you remain irrevocably human.
Ergo, some of my answers you will understand, and some of them you will not.
Concordantly, while your first question may be the most pertinent, you may or may not realize it is also irrelevant.

Neo: Why am I here?

The Architect: """

gpt2.setMaxOutputLength(500)
generate_with_gpt2(pipeline, the_matrix_script,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
The Architect: Hello, Neo.
Neo: Who are you?

The Architect: I am the Architect. I created the matrix. I've been waiting for you.
You have many questions, and although the process has altered your consciousness, you remain irrevocably human.
Ergo, some of my answers you will understand, and some of them you will not.
Concordantly, while your first question may be the most pertinent, you may or may not realize it is also irrelevant.

Neo: Why am I here?

The Architect:  I wanted to tell you. The matrix is a powerful force in creation. It has brought together the various human races that comprise today's human race.
But you are only one of millions of sentient beings I have created for all that has transpired during history, all those times when human beings did

,id,text,result
0,0,"\nThe Architect: Hello, Neo.\nNeo: Who are you...","[ Is it true that \nThe Architect: Hello, Neo...."
